In [1]:
#USE RED MARKER CAP FOR TESTING
import numpy as np
import cv2
import argparse
from collections import deque


cap=cv2.VideoCapture(0) #create a video capture object with the connected camera

pts = deque(maxlen=64) #create a deque object (list-like object) with max length of 64

Lower_red = np.array([0, 100, 0]) # lower boundry of color in hsv that wil be tracked in image
Upper_red = np.array([10, 255, 255])# upper boundry of color in hsv that will be tracked in image
#figured out true red color through http://hanzratech.in/2015/02/07/caveat-thresholding-hue-component.html

while True:
    ret, img=cap.read() # get the current frame and put it into image, ret can be true or false, if a frame was received
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #convert img from bgr(blue,green,red) color to hsv(hue, sat, value) color
    kernel=np.ones((5,5),np.uint8) #fill an 5x5 array(matrix) with zeroes of type unsigned int 8
    mask = cv2.inRange(hsv,Lower_red,Upper_red) #creates a mask containing only the plotting of pixels that are between the red ranges
    mask = cv2.erode(mask, kernel, iterations=2) #make sure that only the marker is seen
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel) #opens a morphology, so that a dilation can be created
    #mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    mask = cv2.dilate(mask, kernel, iterations=1) #dilate mask, which means to superimpose the 5x5 for every pixel in mask
    res=cv2.bitwise_and(img,img,mask=mask) #create a new image that shows both marker and it's plotting as result
    cnts,heir=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:] #get external contours and only their end points(vertex points)
    center = None 
     
    
    if len(cnts) > 0: #checks if any contours have been found in the mask
        c = max(cnts, key=cv2.contourArea) #compares contours based on contour area
        ((x, y), radius) = cv2.minEnclosingCircle(c) #makes the minimum possible circle based on comparisson
        M = cv2.moments(c) #calculate moments( weighted average of the image pixels' intensities) of comparisson and returns a matrix
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))#calculate center
         
        #if radius is more than 5, draw a circle around the center and one in the center itself    
        if radius > 5:
            cv2.circle(img, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
            
    
    pts.appendleft(center)# que acts as the line, which can hold only 64 particles, which fade after new ones are added
    for i in range (1,len(pts)):
        if pts[i-1]is None or pts[i] is None:
            continue
        #create line that connects between center and previous center
        thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
        cv2.line(img, pts[i-1],pts[i],(0,0,225),thick)
    

    cv2.imshow("Frame", img) #show image
    cv2.imshow("mask",mask) #show mask
    cv2.imshow("res",res) #show result


    k=cv2.waitKey(30) & 0xFF #check if space is clicked and if yes, finish iteration
    if k==32:
        break
# cleanup the camera and close any open windows
cap.release()
cv2.destroyAllWindows()